In [1]:
import pandas as pd
import random
from collections import Counter
import numpy as np
# !pip3 install -q sklearn
from sklearn.linear_model import LinearRegression
# import tensorflow as tf
# import tensorflow.keras as keras
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, LSTM, Dropout

In [2]:
xcelLocation = "../xcels/"
# !export xcelLocation="../xcels/"
csvFileName = "MarketWatchPlus-1398_5_7"
csvFileName = "1382-2-2"
RECORD_THRESHOLD = 3000
TESTCASE_NUMBER = 20
alreadyPrep = False

In [3]:
!ls $xcelLocation | grep ".csv" > csvFiles
tmp = !cat csvFiles
csvFiles = [name[:-4] for name in tmp]

In [4]:
def readCsv(xcelLocation, csvFileName):
    df=pd.read_csv(xcelLocation + csvFileName + '.csv', sep=',',header=[0])
    return df

def addDiff(df): 
    diffTable = df.loc[:,["symbol", "min", "max"]]
    difs = []
    for row in range(len(diffTable)):
        minim = diffTable.at[row, "min"]
        maxim = diffTable.at[row, "max"]
        if(minim != 0):
            difs.append((maxim - minim)/minim * 100)
        else:
            difs.append(0)
    diffTable.loc[:,"diff"] = difs
    df.loc[:,"diff"] = difs
    return df, diffTable

In [5]:
def prepare():
    global alreadyPrep
    global df
    global testSymbol
    
    if alreadyPrep:
        return
    alreadyPrep = True
    
    df0 = readCsv(xcelLocation=xcelLocation, csvFileName="master0")
    df1 = readCsv(xcelLocation=xcelLocation, csvFileName="master1")
    df = pd.concat([df0, df1], ignore_index=True)
    df,_ = addDiff(df=df)

    allSymbols = df.symbol.tolist()
    symbols = list(set(df.symbol))[1:]
    print("all symbols: ", len(symbols))
    counts = Counter(allSymbols)
    testSymbols = []
    tmpSymbols = []
    for symbol in symbols:
        if counts[symbol] > RECORD_THRESHOLD:
            tmpSymbols.append(symbol)
    for i in range(TESTCASE_NUMBER):
        ran = random.randint(0, len(tmpSymbols)-1)
        testSymbols.append(tmpSymbols[ran])
        tmpSymbols.remove(tmpSymbols[ran])
    print("test symbol", len(testSymbols))

In [6]:
def runSimulator(testDF, minMaxPredictFunction):
    ans = testDF.iloc[len(testDF) - 1]
    testDF = testDF.drop(len(testDF) - 1)
    predictMin, predictMax = minMaxPredictFunction(testDF)

    result = 0
    if predictMin >= ans["min"] and predictMin <= ans["max"]:
        if predictMax <= ans["max"] and predictMax >= ans["min"]:
            result += 1
        else:
            result += 2
    return result

In [7]:
def calcRate(minMaxPredictFunction):
    prepare()
    SS = 0
    SF = 0
    FF = 0
    
    for symbol in testSymbols:
        testDF = df.loc[df["symbol"] == symbol].reset_index(drop=True)
        for i in range(200):
            result = runSimulator(testDF, minMaxPredictFunction)
            if result % 2 == 1:
                SS += 1
            if (result/2) % 2 == 1:
                SF += 1
            if result == 0:
                FF += 1
            testDF = testDF.drop([len(testDF) - 1, len(testDF) - 2, len(testDF) - 3, len(testDF) - 4])
    print("ss, sf, ff: ", SS, " ", SF, " ", FF)
    print("prec ", SS/(SS + SF))
    print("rec ", (SS + SF)/(SS + FF + SF))
            

In [8]:
def rator1(inputDF):
    size = len(inputDF)
    return inputDF.iloc[size - 1]["min"]*102/100 , inputDF.iloc[size - 1]["max"]

In [9]:
def rator2(inputDF):
    regressor = Sequential()
    regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = 50))
    regressor.add(Dropout(0.2))

    regressor.add(Dense(units = 1))

    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

    regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

In [10]:
# calcRate(rator1)

In [48]:
def alarm1():
    prepare()
    df98 = df.loc[df["year"] == 1398].reset_index(drop=True)
    hit5 = df98.loc[
#                     (df98["diff"] <= 10) & (df98["diff"] > 9.8) & 
                    (df98["ending-percent"] >= 4) & (df98["month"] == 6) &
                    True ].reset_index(drop=True)
    return hit5

In [51]:
hitdf = alarm1()
hitAllSymbols = hitdf.symbol.tolist()
hitSymbols = list(set(hitdf.symbol))[1:]

counts = Counter(hitAllSymbols)
for symbol in hitSymbols:
    if counts[symbol] > 3:
        print(symbol)
# hitdf

سفانو
كفرآور
وبرق
سصفها
غفارس
قاسم
لكما
سخواف
دابور
كاسپينح
واحصا
حآسا
ولانا
فسازان
نمرينو
كفرا
